In [ ]:
import cv2
import numpy as np


## grabs frame from camera one or two
def grabCam(camNum):
    ## cam indexes for camera number
    cam1 = 0
    cam2 = 1
    if(camNum==1):
        cam = cv2.VideoCapture(cam1)
        ret, frame = cam.read()    
        print(ret)
        cam.release()
        return frame
    if(camNum==2):
        cam = cv2.VideoCapture(cam2)
        ret, frame = cam.read()    
        print(ret)
        cam.release()
        return frame
 
## shows a frame
def showPic(fram):
    ## Show frame
    cv2.imshow("image",fram)
    k= cv2.waitKey(0)
    if k==27:
        cv2.destroyAllWindows()
    ## Deactivate Program
    cv2.destroyAllWindows()

## finds circle with color mask
def findCircle(frame):
    ## color mask
    lower = np.array([90,40,200])
    upper = np.array([140,100,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)

    ## contour detection
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
              return [x,y,w,h]
    return -1
## finds square with color mask
def findSquare(frame):
    ## color mask
    lower = np.array([0,80,175])
    upper = np.array([15,200,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)
    
    ## contour detection
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
                epsilon = 0.02 * cv2.arcLength(contour,True)
                approx = cv2.approxPolyDP(contour,epsilon,True)
                print(approx)
                max1 = approx[0]
                max2 = approx[1]
                for i in range(len(approx)):
                    if approx[i][0][1]<max1[0][1] and max1[0][1]>max2[0][1] and not np.array_equal(approx[i],max2):
                        max1 = approx[i]
                    elif approx[i][0][1]<max2[0][1] and not np.array_equal(approx[i],max1):
                        max2 = approx[i]
                print("final",max1)
                print(max2)
                angle = math.degrees(math.atan((max2[0][0]-max1[0][0])/(max2[0][1]-max1[0][1])))
                return [x,y,w,h,angle]
    return -1


## returns list with [x,y,w,h] for circle, triangle, square, star, cross
def findBlocks(frame):
    circle = findCircle(frame)
    square = findSquare(frame)
    triangle, cross, star = findTriangleCrossStar(frame)
    return circle,triangle,square,star,cross

def findRobot(frame):
    ## sometimes robot area is smaller or splits into two contours so we will always pick the largest area
    ## initialize 
    robot = [0,0,0,0]
    area = 20
    ## lower and upper color bounds (hue,sat,val)
    ## hue maxes at 179 others 255
    lower = np.array([105,230,0])
    upper = np.array([120,255,255])
    
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame = cv2.inRange(frame,lower,upper)
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    ## look at each contour
    for contour in contours:
    ## to filter out noise make sure area over a certain ammount - try 800
        if cv2.contourArea(contour) >area:
            ## coordinates of top left contour
            x,y,w,h = cv2.boundingRect(contour)
            if x>400 and x<1500 and w*h>robot[2]*robot[3]:
                print("Robot",x,y,w,h)
                robot = [x,y,w,h]

    print("Final Robot",robot[0],robot[1],robot[2],robot[3])
    return robot

## shape is given by list [x,y,w,h]
def findMiddle(shape):
    x = shape[0]+(int(shape[2]/2))
    y = shape[1]+(int(shape[3]/2))
    return [x,y]

In [ ]:
## servos plug in black away from robot
## check port in terminal with 
import serial
import time
import math

## y and z should not be negative

## turns degrees into microseconds for serial command
def angleToMicro(angle):
    return round(angle*11.11111)+500
def microToAngle(micro):
    return round((micro-500)/11.11111)

## Does the inverse kinematics
## returns the microseconds in order
def findServos(curr):
    ##initialize x,y,z
    x = curr[0]
    y = curr[1]
    z = curr[2]
    
    ##
    ## servo 0
    ##
    if x==0:
        degree = 90
    if x>0:
        degree = math.degrees(math.atan(y/x))
    if x<0:
        degree = 180 - math.degrees(math.atan(y/(x*-1)))
    serv0 = angleToMicro(degree)
    
    ##
    ## servo 1 and 2
    ##
    
    ## measurements
    beginArm = 3
    armOne = 6
    armTwo = 7.5
    armThree = 4
    ## find "x"
    v = math.sqrt(x**2+y**2)
    ## find "y"
    z = z+(armThree-beginArm)
    ## find the two possible angles
    servTwo_plus = math.atan2(math.sqrt(1-((v**2+z**2-armOne**2-armTwo**2)/(2*armOne*armTwo))**2),(v**2+z**2-armOne**2-armTwo**2)/(2*armOne*armTwo))
    servTwo_minus = math.atan2(-1*(math.sqrt(1-((v**2+z**2-armOne**2-armTwo**2)/(2*armOne*armTwo))**2)),(v**2+z**2-armOne**2-armTwo**2)/(2*armOne*armTwo))
    if servTwo_plus>0:
        radTwo = servTwo_plus
    if servTwo_minus>0:
        radTwo = servTwo_minus
    k1 = armOne+(armTwo*math.cos(radTwo))
    k2 = armTwo*math.sin(radTwo)
    degreeOne = math.degrees(math.atan2(z,v)+math.atan2(k2,k1))
    degreeTwo = math.degrees(radTwo)

    ##
    ## servo 3
    ##

    
    serv3 = 180 - (degreeOne + (180-degreeTwo))
    serv3 = angleToMicro(serv3) 
    serv1 = angleToMicro(degreeOne)
    serv2 = angleToMicro(degreeTwo)
    

    return serv0,serv1,serv2,serv3

## cleans up findServos and returns command line not servos
def getSerialCommand(curr,ser4,angle):
    ser0,ser1,ser2,ser3 = findServos(curr)
    angle0 = microToAngle(ser0)
    if ser4 ==-1:
        ser4 = ser0
    if angle!=-1:
        if angle<0:
            ser4 = angleToMicro(angle0+90+angle)
        else:
            ser4 = angleToMicro(angle0-90+angle)
    return f"#0 P{ser0} T1500 #3 P{ser3} T1500 #1 P{ser1} T1500 #2 P{ser2} T1500 #4 P{ser4} T1500 \r"

## sends serial
def sendSerial(commandString):
    port = "/dev/tty.usbserial-0001"
    baud = 38400
    try:
        ser = serial.Serial(port,baud,timeout=3)
    except:
        print("error opening serial")
    ser.write(commandString.encode('UTF-8'))
    time.sleep(2.5)
    ser.close()
    

## unit in inches, curr is list [x,y,z] , serv4 is optional do -1 to ignore
def moveUnitDirect(unit,direction,curr,serv4,angle):
    if direction =="x":
        curr = [curr[0]+unit,curr[1],curr[2]]
        command = getSerialCommand(curr,serv4,angle)
        sendSerial(command)
    elif direction =="y":
        curr = [curr[0],curr[1]+ unit,curr[2]]
        command = getSerialCommand(curr,serv4,angle)
        sendSerial(command)
    elif direction =="z":
        curr = [curr[0],curr[1],curr[2]+unit]
        command = getSerialCommand(curr,serv4,angle)
        sendSerial(command)
    return curr

In [ ]:
import math
def findTriangleCrossStar(frame):
    ## initialize star and cross to -1 
    star = -1
    cross = -1
    maybeCross = -1
    maybeCrossA = -1
    maybeStar = -1
    maybeStarA = -1
    triangle = -1 
    ## color mask
    lower = np.array([15,10,220])
    upper = np.array([60,150,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)
    showPic(frame)
    ## contour detection
    
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
                epsilon = 0.02 * cv2.arcLength(contour,True)
                approx = cv2.approxPolyDP(contour,epsilon,True)
                print("1 ",len(approx))
                if len(approx) == 12 or len(approx)==13:
                    cross = [x,y,w,h]
                    continue
                if len(approx) == 11:
                    maybeCross = [x,y,w,h]
                    continue
                epsilon = 0.07 * cv2.arcLength(contour,True)
                approx = cv2.approxPolyDP(contour,epsilon,True)
                print("2 ",len(approx))
                if len(approx) == 5 or len(approx)==6:
                    star = [x,y,w,h]
                    starA = approx
                    continue
                if len(approx) == 4:
                    maybeStar = [x,y,w,h]
                    maybeStarA = approx
                    continue
                if len(approx) == 3:
                    triangle = [x,y,w,h]
                    triangleA = approx
    if cross ==-1:
        cross = maybeCross
    if star==-1 and cross!=maybeCross:
        star = maybeCross
    if star==-1:
        star = maybeStar
    if triangle == -1:
        triangle = maybeStar
        triangleA = maybeStarA
    print(triangle)
    ## triangle angle
    max1 = triangleA[0]
    max2 = triangleA[1]
    for i in range(len(triangleA)):
        if triangleA[i][0][1]<max1[0][1] and max1[0][1]>max2[0][1] and not np.array_equal(triangleA[i],max2):
            max1 = triangleA[i]
        elif triangleA[i][0][1]<max2[0][1] and not np.array_equal(triangleA[i],max1):
            max2 = triangleA[i]
    print("final",max1)
    print(max2)
    angle = math.atan((max2[0][0]-max1[0][0])/(max2[0][1]-max1[0][1]))
    print(math.degrees(angle))
    triangle.append(math.degrees(angle))
    return triangle, cross, star

In [ ]:
import time
def grabBlock(blockMid,ajustY,ajustX,angle):
    print(ajustY)
    print(ajustX)
    ## move robot to start pos
    position = [0,7.5,5]
    command = getSerialCommand(position,-1,angle)
    sendSerial(command)
    #####
    ##### Considering the y direction
    #####

    ## find robot cam position
    frame = grabCam(1)
    robot = findRobot(frame)

    ## move one unit 
    position = moveUnitDirect(1,"y",position,-1,angle)

    ## find the robot new position
    frame = grabCam(1)
    newRobot = findRobot(frame)

    ## distance moved by 1 unit
    dist = robot[0]-newRobot[0]
    

    ## find new unit to reach shape
    unit = (newRobot[0]-blockMid[0]+ajustY)/dist

    ## set new robot mid to current
    robot = newRobot

    while unit>0.025  or unit<-0.025:
        unit = unit
        ## going to cap at 3 units at a time to prevent as much error as possible
        if unit >=3:
            unit = 3
        if unit<=-3:
            unit = -3 
    
        ## move unit
        position = moveUnitDirect(unit,"y",position,-1,angle)
    
        ## find the robot new position
        frame = grabCam(1)
        newRobot = findRobot(frame)
    
    
        ## distance moved per one unit averaged with new and old dist value
        dist = (robot[0]-newRobot[0])/unit

        ## find new unit to reach shape
        if dist ==0:
            unit = 0
        else:
            unit = (newRobot[0]-blockMid[0]+ajustY)/dist
    print("DONE")

    #####
    ##### Considering the x direction
    #####

    ## find robot cam position
    frame = grabCam(1)
    robot = findRobot(frame)

    ## get center points
    robotMid = findMiddle(robot)

    ## move one unit 
    position = moveUnitDirect(1,"x",position,-1,angle)

    ## find the robot new position
    frame = grabCam(1)

    ## get center points
    newRobotMid = findMiddle(findRobot(frame))

    ## distance moved by 1 unit
    dist = robotMid[1]-newRobotMid[1]

    ## find new unit to reach shape
    unit = (newRobotMid[1]-blockMid[1]+ajustX)/dist

    ## set new robot mid to current
    robotMid = newRobotMid

    while unit>0.025 or unit<-0.025: 
        ## move unit
        position = moveUnitDirect(unit,"x",position,-1,angle)
    
        ## find the robot new position
        frame = grabCam(1)
        newRobot = findRobot(frame)
        if newRobot == [0,0,0,0]:
            break
        newRobotMid = findMiddle(newRobot)
        ## distance moved per one unit averaged with new and old dist value
        dist = (robotMid[1]- newRobotMid[1])/unit

        ## find new unit to reach shape
        if dist == 0:
            unit = 0
        else:
            unit = (newRobotMid[1]-blockMid[1]+ajustX)/dist
    print("DONE")

    ## open gripper
    command = "#5 P1500 T500 \r"
    sendSerial(command)

    ## go down to block height
    position = moveUnitDirect(-3.5,"z",position,-1,angle)

    ## close gripper
    command = "#5 P500 T500 \r"
    sendSerial(command)

    return position
def grabTriangle(triangle,angle):
    ## calculate X adjustment
    triangleMid = findMiddle(triangle)
    print(triangleMid)
    ajustX = (-0.17*triangleMid[1])+120
    if triangleMid[1]>650:
        ajustX=ajustX-15
    elif triangleMid[1]>550:
        ajustX=ajustX-8
    if triangleMid[1]<450:
        ajustX=ajustX+10
    ## calculate Y ajustment
    if triangleMid[0]>1100:
        ajustY = 0
    else:
        ajustY = 40

    if angle <= -60:
        ajustY+=5
        ajustX-=5
        print("plus 5")
    elif angle<=-50:
        ajustY+=10
        ajustX-=10
        print("plus 10")
    elif angle<=0:
        ajustY+=15
        print("plus 15")
        ajustX-=15
    elif angle<=65:
        print("sub 15")
        ajustY-=15
        ajustX-=15
    elif angle<=75:
        ajustY-=10
        ajustX-=10
        print("sub 10")
    elif angle<=80:
        print("sub 5")
        ajustY-=5
        ajustX-=5
    
    ## grab circle
    return grabBlock(triangleMid,ajustY,ajustX,angle)

def placeTriangle(position):
    ## go up to travel height
    position = moveUnitDirect(3.5,"z",position,-1,-1)

    ## put in triangle hole
    position = [0,7.5,8]
    command = getSerialCommand(position,-1,-1)
    sendSerial(command)

    position = [-7,3.5,7.5]
    command = getSerialCommand(position,1700,-1)
    sendSerial(command)
    
    position = [-6.75,2.8,6.5]
    command = getSerialCommand(position,1650,-1)
    sendSerial(command)

    command = "#5 P1500 T1000 \r"
    sendSerial(command)
    
    position = [-7,3.5,7.5]
    command = getSerialCommand(position,1700,-1)
    sendSerial(command)

    position = [0,7.5,7]
    command = getSerialCommand(position,-1,-1)
    sendSerial(command)


In [ ]:
time.sleep(2)
## start robot but get out of way to find blocks
position = [0,7,5,5]
command = getSerialCommand(position,-1,-1)
sendSerial(command)
position = [7,1,5]
command = getSerialCommand(position,-1,-1)
sendSerial(command)

## get top cam
frame = grabCam(1)

## find the shapes
circle,triangle,square,star,cross = findBlocks(frame)

if triangle ==-1:
    frame = grabCam(1)
    triangle,cross,star = findTriangleCrossStar(frame) 

position = grabTriangle(triangle[:4],triangle[4])
position = placeTriangle(position)

position = [0,7,5,5]
command = getSerialCommand(position,-1,-1)
sendSerial(command)
position = [7,1,5]
command = getSerialCommand(position,-1,-1)
sendSerial(command)
"""
    ## get top cam
    frame = grabCam(1)
    triangle,cross,star = findTriangleCrossStar(frame)
"""